In [ ]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Iris dataset 불러오기
X, y = datasets.load_iris(return_X_y=True)

# Train, Test 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 모델 하이퍼파라미터 정의
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# 모델 학습
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# test 데이터셋 Predict
y_pred = lr.predict(X_test)

# Metrics 계산
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# Tracking Server 설정
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000") # 환경 변수를 설정했지만, 확실하게 설정

# 새로운 MLFlow 실험을 생성
mlflow.set_experiment("MLflow Quickstart")

# MLflow 실험 시작
with mlflow.start_run():
    # 하이퍼파라미터 로그
    mlflow.log_params(params)

    # loss, metric 로그
    mlflow.log_metric("accuracy", accuracy)

    # 태그 설정 (실행에 메모 남기기)
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # 입출력 형태 추론
    signature = infer_signature(X_train, lr.predict(X_train))

    # 모델 로그
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

In [ ]:
# 예측을 위해 일반 Python 함수 모델(pyfunc)로 모델을 다시 불러옵니다.
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]